In [7]:
import os
import shutil
import yaml
from glob import glob
from collections import Counter
from ultralytics import YOLO

In [8]:
# --- 1. 기본 경로 설정 ---
# 이전에 다운로드한 두 데이터셋의 상위 폴더 경로를 정확하게 지정해주세요.
# 예: 'C:/Users/Admin/work space/2nd/'
BASE_DATA_PATH = 'C:/Users/Admin/work space/2nd/'

# Roboflow에서 다운로드했을 때 생성된 폴더 이름
# (필요시 실제 폴더 이름으로 수정)
DATASET1_NAME = 'acne04-1'
DATASET2_NAME = 'Acne-detection-1'

In [9]:
# 최종 데이터셋이 저장될 폴더 이름
COMBINED_DATASET_NAME = 'acne_dataset_final'

In [10]:
# --- 2. 클래스 목록 정의 ---
# 이전 단계에서 확인된 각 데이터셋의 실제 클래스 목록
ORIGINAL_D1_NAMES = ['fore', 'papule', 'pustule', 'nodule', 'whiteheads', 'cyst', 'blackheads']
ORIGINAL_D2_NAMES = ['comedone', 'pustule', 'papule', 'cyst', 'nodule']

# 두 목록을 합치고 중복을 제거한 뒤, 알파벳 순으로 정렬하여 최종 클래스 목록 생성
final_class_set = set(ORIGINAL_D1_NAMES) | set(ORIGINAL_D2_NAMES)
FINAL_CLASS_NAMES = sorted(list(final_class_set))

print("✅ 최종 통합 클래스 목록 (총 {}개):".format(len(FINAL_CLASS_NAMES)))
print(FINAL_CLASS_NAMES)
print("-" * 50)


✅ 최종 통합 클래스 목록 (총 8개):
['blackheads', 'comedone', 'cyst', 'fore', 'nodule', 'papule', 'pustule', 'whiteheads']
--------------------------------------------------


In [11]:
# --- 3. 클래스 ID 재매핑 테이블 생성 ---
# 최종 클래스 이름을 기준으로 새로운 ID 부여 (name -> id)
final_name_to_id = {name: i for i, name in enumerate(FINAL_CLASS_NAMES)}

# 각 데이터셋의 원래 ID를 -> 최종 ID로 변환하는 테이블 생성
# 데이터셋 1 매핑 테이블
original_d1_name_to_id = {name: i for i, name in enumerate(ORIGINAL_D1_NAMES)}
remap_d1 = {orig_id: final_name_to_id[name] for name, orig_id in original_d1_name_to_id.items()}

# 데이터셋 2 매핑 테이블
original_d2_name_to_id = {name: i for i, name in enumerate(ORIGINAL_D2_NAMES)}
remap_d2 = {orig_id: final_name_to_id[name] for name, orig_id in original_d2_name_to_id.items()}


In [12]:
# --- 4. 데이터 병합 및 라벨 재매핑 실행 ---
src_path_d1 = os.path.join(BASE_DATA_PATH, DATASET1_NAME)
src_path_d2 = os.path.join(BASE_DATA_PATH, DATASET2_NAME)
combined_path = os.path.join(BASE_DATA_PATH, COMBINED_DATASET_NAME)

# 기존에 폴더가 있다면 삭제하고 새로 시작
if os.path.exists(combined_path):
    shutil.rmtree(combined_path)
    print(f"기존 '{combined_path}' 폴더를 삭제했습니다.")

print(f"'{combined_path}' 폴더를 생성하고 데이터 병합을 시작합니다...")

# 데이터셋을 처리하는 함수
def process_dataset(src_path, remap_dict, dest_path):
    for split in ['train', 'valid', 'test']:
        src_label_dir = os.path.join(src_path, split, 'labels')
        src_image_dir = os.path.join(src_path, split, 'images')
        
        dest_label_dir = os.path.join(dest_path, split, 'labels')
        dest_image_dir = os.path.join(dest_path, split, 'images')
        
        os.makedirs(dest_label_dir, exist_ok=True)
        os.makedirs(dest_image_dir, exist_ok=True)
        
        if not os.path.exists(src_label_dir):
            continue

        # 라벨 파일 처리 및 이미지 복사
        for label_filename in os.listdir(src_label_dir):
            if not label_filename.endswith('.txt'):
                continue

            # 라벨 파일 재매핑
            with open(os.path.join(src_label_dir, label_filename), 'r') as f_in:
                lines = f_in.readlines()
            
            with open(os.path.join(dest_label_dir, label_filename), 'w') as f_out:
                for line in lines:
                    parts = line.strip().split()
                    if not parts:
                        continue
                    
                    original_id = int(parts[0])
                    if original_id in remap_dict:
                        parts[0] = str(remap_dict[original_id])
                        f_out.write(' '.join(parts) + '\n')

            # 해당 이미지 파일 복사
            base_filename = os.path.splitext(label_filename)[0]
            for ext in ['.jpg', '.jpeg', '.png']:
                image_filename = base_filename + ext
                src_image_path = os.path.join(src_image_dir, image_filename)
                if os.path.exists(src_image_path):
                    shutil.copy(src_image_path, os.path.join(dest_image_dir, image_filename))
                    break

# 각 데이터셋 처리 실행
process_dataset(src_path_d1, remap_d1, combined_path)
process_dataset(src_path_d2, remap_d2, combined_path)

print("✅ 데이터 병합 및 라벨 재매핑 완료!")
print("-" * 50)

기존 'C:/Users/Admin/work space/2nd/acne_dataset_final' 폴더를 삭제했습니다.
'C:/Users/Admin/work space/2nd/acne_dataset_final' 폴더를 생성하고 데이터 병합을 시작합니다...
✅ 데이터 병합 및 라벨 재매핑 완료!
--------------------------------------------------


In [13]:
# --- 5. 최종 data.yaml 파일 생성 ---
yaml_path = os.path.join(combined_path, 'data.yaml')

yaml_data = {
    'train': os.path.join('train', 'images'),
    'val': os.path.join('valid', 'images'),
    'test': os.path.join('test', 'images'),
    'nc': len(FINAL_CLASS_NAMES),
    'names': FINAL_CLASS_NAMES
}

with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_data, f, allow_unicode=True, sort_keys=False)

print(f"✅ '{yaml_path}' 파일 생성 완료!")
print("--- data.yaml 내용 ---")
print(yaml.dump(yaml_data, allow_unicode=True, sort_keys=False))
print("-" * 50)

✅ 'C:/Users/Admin/work space/2nd/acne_dataset_final\data.yaml' 파일 생성 완료!
--- data.yaml 내용 ---
train: train\images
val: valid\images
test: test\images
nc: 8
names:
- blackheads
- comedone
- cyst
- fore
- nodule
- papule
- pustule
- whiteheads

--------------------------------------------------


In [14]:

print("🚀 모델 학습을 시작합니다...")

# 사전 학습된 YOLOv8n 모델 로드
model = YOLO('yolov8n.pt')

# 모델 학습 실행
results = model.train(
    data=yaml_path,
    epochs=200,      # 필요에 따라 에포크 수 조정
    imgsz=640,
    batch=16,
    project=BASE_DATA_PATH, # 결과 저장 경로
    name='acne_model_8_classes' # 학습 결과가 저장될 폴더 이름
)

print("\n🎉 모든 과정이 완료되었습니다! 학습 결과는 다음 폴더에 저장되었습니다:")
print(os.path.join(BASE_DATA_PATH, 'acne_model_8_classes'))


🚀 모델 학습을 시작합니다...
New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/Admin/work space/2nd/acne_dataset_final\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, m

In [15]:
from ultralytics import YOLO
import os

# 1. 가장 성능이 좋았던 모델 파일 경로
MODEL_PATH = 'C:/Users/Admin/work space/2nd/acne_model_8_classes/weights/best.pt'

# 2. 학습 시 생성된 data.yaml 파일 경로
DATA_YAML_PATH = 'C:/Users/Admin/work space/2nd/acne_dataset_final/data.yaml'

# 모델 로드
model = YOLO(MODEL_PATH)

# 모델 성능 검증 (test 데이터셋 사용)
# split='val'로 변경하면 validation 데이터셋으로 검증합니다.
print("🚀 테스트 데이터셋으로 모델 성능을 검증합니다...")
metrics = model.val(data=DATA_YAML_PATH, split='test', imgsz=640)

print("\n--- 최종 성능 지표 ---")
print(f"mAP50-95 (평균 IoU 0.5~0.95): {metrics.box.map:.4f}")
print(f"mAP50 (IoU 0.5 기준): {metrics.box.map50:.4f}")
print(f"mAP75 (IoU 0.75 기준): {metrics.box.map75:.4f}")

🚀 테스트 데이터셋으로 모델 성능을 검증합니다...
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 2.21.3 MB/s, size: 16.4 KB)
val: Scanning C:\Users\Admin\work space\2nd\acne_dataset_final\test\labels... 123 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 123/123 213.4it/s 0.6s0.1s
val: New cache created: C:\Users\Admin\work space\2nd\acne_dataset_final\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.2it/s 6.5s0.5ss
                   all        123        461      0.477      0.451      0.419      0.174
              comedone         24         86        0.6      0.605      0.599      0.253
                  cyst         18         19      0.669      0.639      0.701       0.28
                nodule         19         19      0.622    

In [16]:
import matplotlib.pyplot as plt
import cv2
import os

# 학습 결과가 저장된 폴더 경로
RESULTS_DIR = 'C:/Users/Admin/work space/2nd/acne_model_8_classes/'

# (1) 혼동 행렬 (Confusion Matrix) 시각화
# 모델이 어떤 클래스를 다른 클래스로 착각하는지 보여줍니다.
# 대각선에 값이 집중될수록 좋은 모델입니다.
img_cm_path = os.path.join(RESULTS_DIR, 'confusion_matrix.png')
if os.path.exists(img_cm_path):
    img_cm = cv2.imread(img_cm_path)
    plt.figure(figsize=(12, 10))
    plt.imshow(cv2.cvtColor(img_cm, cv2.COLOR_BGR2RGB))
    plt.title('Confusion Matrix', fontsize=20)
    plt.axis('off')
    plt.show()
else:
    print(f"'{img_cm_path}' 파일을 찾을 수 없습니다.")

# (2) 검증 데이터 예측 결과 시각화
# Validation 데이터셋의 일부 이미지에 대한 예측 결과입니다.
img_pred_path = os.path.join(RESULTS_DIR, 'val_batch0_pred.jpg')
if os.path.exists(img_pred_path):
    img_pred = cv2.imread(img_pred_path)
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img_pred, cv2.COLOR_BGR2RGB))
    plt.title('Validation Predictions', fontsize=20)
    plt.axis('off')
    plt.show()
else:
    print(f"'{img_pred_path}' 파일을 찾을 수 없습니다.")

<Figure size 1200x1000 with 1 Axes>

<Figure size 1500x1000 with 1 Axes>